# Imports

In [1]:
import pandas as pd

In [2]:
import nltk
import re

from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\mkavalio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
import numpy as np
# def dummy_npwarn_decorator_factory():
#   def npwarn_decorator(x):
#     return x
#   return npwarn_decorator
# np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [5]:
from bertopic import BERTopic

c:\Users\mkavalio\codedir\TextAnalysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import pipeline #if error change version of transformers

c:\Users\mkavalio\codedir\TextAnalysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


# Dataset import and text preprocessing

In [10]:
df = pd.read_csv(r"C:\Users\mkavalio\codedir\TextAnalysis\Data\Car_Reviews_Database_KaggleSentiment.csv", encoding='ISO-8859-1')
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 234 entries, 0 to 277
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    234 non-null    int64 
 1   Model   234 non-null    object
 2   Review  234 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [11]:
def preprocess_text(text: str) -> str:
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text) # leave only abc characters and spaces
    
    # remove stopwords
    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if not w.lower() in [stopwords]]
    text = " ".join(tokens)
    text = text.lower().strip() # removes leading and trailing whitespaces
    
    return text

In [12]:
df['Review_processed'] = df['Review'].apply(lambda text: preprocess_text(text))

In [13]:
df.sample(2)

,Year,Model,Review,Review_processed
2,2010,Honda,I have owed and driven Honda products for 20 y...,i have owed and driven honda products for year...
277,2019,BMW,"The BMW 3 Series is known as a driver's car, a...",the bmw series is known as a driver s car and ...


In [14]:
docs = df['Review_processed'].tolist()

# Experiments

## BERTopic

In [18]:
topic_model = BERTopic()


In [19]:
topics, probs = topic_model.fit_transform(docs)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/53aa51172d142c89d9012cce15ae4d6cc0ca6895895114379cacb4fab128d9db?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1738936470&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczODkzNjQ3MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9zZW50ZW5jZS10cmFuc2Zvcm1lcnMvYWxsLU1pbmlMTS1MNi12Mi81M2FhNTExNzJkMTQyYzg5ZDkwMTJjY2UxNWFlNGQ2Y2MwY2E2ODk1ODk1MTE0Mzc5Y2FjYjRmYWIxMjhkOWRiP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=aOEuKzHrN27vMuwS~JrJ2IwcAs9cIW6c7lxGOY7-VwDMN0cv3wdvxZZNHjXKAmeF-mgiEKYihL5zGDjv3xENtzeDfXIgA5HMCpqr7bQVth0ANs6ONwMfsaCVKiRp-Hw2i4U~39XN2wSC4R0R~7ZkfK5ML9di-QfJyifMZcjDBNX~hvmeIL1M8zmNf70nEq4L-J2zplntejkvCGFYzWhx9LLjFS4bM0hCRZZs4l59ZlV4~5ZaoNUAI~R9NCAwujbept~OVb0kmsyQ0dX6Uivzd9mE~mHHivFQ7iycdMcUjON